<a href="https://colab.research.google.com/github/td-yoshimura/hivemall-notebooks/blob/master/notebooks/%5BEN%26JP%5Dhivemall101_with_churn_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary / 概要

By this notebook, you can upload churn datasets into your Treasure Data account briefly.  
Please refer [this article](https://aws.amazon.com/blogs/machine-learning/predicting-customer-churn-with-amazon-machine-learning/)  
Beforehand, you only have to 
- prepare UserName and Password
- prepare the TD API key
- create database  
In addition, this notebook shows how to run a machine learning task with churn dataset.

<br>

[Churnデータセット](https://aws.amazon.com/blogs/machine-learning/predicting-customer-churn-with-amazon-machine-learning/) をTreasure Data環境にアップロードして、解約者の予測を行います。
事前に必要なものは、
- Treasure Dataのユーザ名とパスワード
- Treasure DataのAPIキー
- Databaseの作成  
のみです。


In [0]:
%env TD_API_KEY = YOUR_TD_API_KEY
%env DATABASE = YOUR_DATABASE

# SetUp / セットアップ

<font color="red"><b>You will be asked to input UserName and Password of Treasure Data. Please fill them.</b></font>
<br>
<font color="red"><b>以下のセルを実行するとTreasure Dataのユーザ名とパスワードの入力を求められるので入力してください。</b></font>

In [0]:
!curl -L https://toolbelt.treasuredata.com/sh/install-ubuntu-trusty-td-agent2.sh | sh
!pip install pandas-td
!td account -f

In [0]:
import os
td_api_key = os.environ['TD_API_KEY']
database = os.environ['DATABASE']
table = 'churn_dataset'

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import seaborn as sns

import pandas_td as td
con = td.connect(apikey=td_api_key)

%matplotlib inline
%load_ext pandas_td.ipython
%td_use $database

import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.max_columns = 30

presto = td.create_engine('presto:{}'.format(database), con=con)
hive = td.create_engine('hive:{}'.format(database), con=con)

# Data Preparation / データの準備

A cell below will create a table on your Treasure Data account.  
This cell takes a little long time (5min.)

<br>

以下のセルを実行すると利用しているTreasure Data環境にテーブルが作成されます。  
５分程度時間がかかります。

In [0]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip
!chmod 777 Data\ sets/churn.txt
!td table:delete -f ${DATABASE} churn_dataset; td table:create ${DATABASE} churn_dataset
!td import:auto \
  --format csv \
  --columns state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,cust_serv_calls,is_churn \
  --column-types string,int,string,string,string,string,int,double,int,double,double,int,double,double,int,double,double,int,double,int,string \
  --auto-create ${DATABASE}.churn_dataset \
  --time-value 1545896745 \
  ./Data\ sets/churn.txt

Check the table.  
テーブルを見てみましょう。

In [0]:
%%td_presto
select
  is_churn,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,cust_serv_calls
from
  churn_dataset
limit 2

各カラムの詳細です。  
https://aws.amazon.com/blogs/machine-learning/predicting-customer-churn-with-amazon-machine-learning/

**Columns**
- state: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- account_length: the number of days that this account has been active
- area_code: the three-digit area code of the corresponding customer’s phone number
- phone: the remaining seven-digit phone number
- intl_plan: whether the customer has an international calling plan: yes/no
- vmail_plan: whether the customer has a voice mail feature: yes/no
- vmail_message: presumably the average number of voice mail messages per month
- day_mins: the total number of calling minutes used during the day
- day_calls: the total number of calls placed during the day
- day_charge: the billed cost of daytime calls
- eve_mins, eve_calls, eve_charge: the billed cost for calls placed during the evening
- night_mins, night_calls, night_charge: the billed cost for calls placed during nighttime
- intl_mins, intl_calls, intl_charge: the billed cost for international calls
- cust_serv_calls: the number of calls placed to Customer Service
- is_churn: whether the customer left the service: true/false

# EDA (Explore Data Analysis)
Before running machine learning tasks, you should understand your data correctly.  
Descriptive statistics and visualization are important for this.

<br>

機械学習の処理を行う前に、データを正しく把握することが重要です。  
そのために記述統計量の確認や可視化を行います。

## Data Size

In [0]:
%%td_presto
select
  count(*)
from
  churn_dataset

## Check if null value exists

In [0]:
%%td_presto
select
  count(is_churn) as is_churn,
  count(state) as state,
  count(account_length) as account_length,
  count(area_code) as area_code,
  count(phone) as phone,
  count(intl_plan) as intl_plan,
  count(vmail_plan) as vmail_plan,
  count(vmail_message) as vmail_message,
  count(day_mins) as day_mins,
  count(day_calls) as day_calls,
  count(day_charge) as day_charge,
  count(eve_mins) as eve_mins,
  count(eve_calls) as eve_calls,
  count(eve_charge) as eve_charge,
  count(night_mins) as night_mins,
  count(night_calls) as night_calls,
  count(night_charge) as night_charge,
  count(intl_mins) as intl_mins,
  count(intl_calls) as intl_calls,
  count(intl_charge) as intl_charge,
  count(cust_serv_calls) as cust_serv_calls
from
  churn_dataset

Null value is not found.  
Nullは含まれていませんでした。

## Target Variable

In [0]:
%%td_presto
select
  is_churn
  ,count(*) as n
from
  churn_dataset
group by
  1

## state(string)

In [0]:
td.read_td_query('''
select
  state
  ,count(*) as n
from
  churn_dataset
group by
  1
order by
  2 desc
''', engine=presto, index_col='state').plot.bar(figsize=(15,5))

## account_length (int)

In [0]:
%%td_presto
select
  avg(account_length) as average
  ,min(account_length) as min
  ,approx_percentile(account_length, 0.25) as pct_25th
  ,approx_percentile(account_length, 0.50) as pct_50th
  ,approx_percentile(account_length, 0.75) as pct_75th
  ,max(account_length) as max
from
  churn_dataset

In [0]:
td.read_td_query('''
  select
    account_length
  from
    churn_dataset
''', engine=presto).hist(bins=20)

## area_code (string)

In [0]:
%%td_presto
select
  area_code
  ,count(*) as n
from
  churn_dataset
group by
  1
order by
  2 desc

## phone(string)
This column is unique for each row. So, this cannot be used for machine learning.  
このカラムは各行に一意の値が割り振られているので、機械学習の説明変数としては使うことができません。

In [0]:
%%td_presto
select
  count(phone) as n
  ,count(distinct phone) as n_unique
from
  churn_dataset

## intl_plan(string)

In [0]:
%%td_presto
select
  intl_plan
  ,count(*) as n
from
  churn_dataset
group by
  1
order by
  2 desc

## vmail_plan(string)

In [0]:
%%td_presto
select
  vmail_plan
  ,count(*) as n
from
  churn_dataset
group by
  1
order by
  2 desc

## vmail_message(int)

In [0]:
td.read_td_query('''
select
  vmail_message
from
  churn_dataset
''', engine=presto).hist(bins=20)

## day_mins(double)

In [0]:
td.read_td_query('''
select
  day_mins
from
  churn_dataset
''', engine=presto).hist(bins=20)

## day_calls(int)

In [0]:
td.read_td_query('''
select
  day_calls
from
  churn_dataset
''', engine=presto).hist(bins=20)

## day_charge(double)

In [0]:
td.read_td_query('''
select
  day_charge
from
  churn_dataset
''', engine=presto).hist(bins=20)

## eve_mins(double)

In [0]:
td.read_td_query('''
select
  eve_mins
from
  churn_dataset
''', engine=presto).hist(bins=20)

## eve_calls(int)

In [0]:
td.read_td_query('''
select
  eve_calls
from
  churn_dataset
''', engine=presto).hist(bins=20)

## eve_charge(double)

In [0]:
td.read_td_query('''
select
  eve_charge
from
  churn_dataset
''', engine=presto).hist(bins=20)

## night_mins(double)

In [0]:
td.read_td_query('''
select
  night_mins
from
  churn_dataset
''', engine=presto).hist(bins=20)

## night_calls(int)


In [0]:
td.read_td_query('''
select
  night_calls
from
  churn_dataset
''', engine=presto).hist(bins=20)

## night_charge(double)

In [0]:
td.read_td_query('''
select
  night_charge
from
  churn_dataset
''', engine=presto).hist(bins=20)

## intl_mins(double)

In [0]:
td.read_td_query('''
select
  intl_mins
from
  churn_dataset
''', engine=presto).hist(bins=20)

## intl_calls(int)


In [0]:
td.read_td_query('''
select
  intl_calls
from
  churn_dataset
''', engine=presto).hist(bins=20)

## intl_charge (double)

In [0]:
td.read_td_query('''
select
  intl_charge
from
  churn_dataset
''', engine=presto).hist(bins=20)

## cust_serv_calls

In [0]:
td.read_td_query('''
select
  cust_serv_calls
  ,count(*) as n
from
  churn_dataset
group by
  1
''', engine=presto, index_col='cust_serv_calls').sort_index().plot.bar()

## Correlations / 相関
With python library, you can visuazlie the relashionship between variables.
<br>
Pythonを利用して、変数間の相関を可視化します。

In [0]:
df = td.read_td_query('''
  select
    case when is_churn like 'True.' then 1 else 0 end as is_churn
    ,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,cust_serv_calls
  from
    churn_dataset    
''', engine=presto)

### Scatter plot / 散布図

In [0]:
pd.plotting.scatter_matrix(df, figsize=(20,15))
plt.show()

### Correlatiion coefficient matrix / 相関行列

In [0]:
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(df.corr(), annot=True, ax=ax, vmin=-0.1, vmax=0.1)
plt.show()

The correlation coefficients of pairs (day_mins, day_charge) , (eve_mins, eve_charge), (intl_mins, intl_charge) equals `1.0`.  
You should remove one of elements from feature variables because of [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity).

<br>

(day_mins, day_charge) , (eve_mins, eve_charge), (intl_mins, intl_charge) の３つのペアは相関係数が `1.0` となっています。  
[多重共線性](https://en.wikipedia.org/wiki/Multicollinearity) のため、各ペアのいずれかの変数は説明変数から除外します。

# Run a machine learning task / 機械学習

## WIthout rescaling / リスケールしないパターン
At first, let's build a model not having features rescaling.    
Rescaling means to have features from 0 to 1.

まずはじめに、説明変数をリスケールせずにモデルを構築します。  
リスケールとは変数を 0 ~ 1に収めるように変換することを指します。

### Preprocess / 前処理

- Data is splitted into training data and test data. 
- String columns are expanded to binary columns.
- Remove unnecessary columns in consideration of [EDA](https://colab.research.google.com/drive/1k062r8-jXhlnU_vjbHCI-Z_yGxt96iE2#scrollTo=CEqJzaJ-rC78)

<br>

- データを学習データと検証データに分けます
- 文字列のカラムはバイナリのカラムに展開します。
- 不要なカラムは除きます。

In [0]:
!td table:delete -f $DATABASE churn_preprocessed_without_rescale; td table:create $DATABASE churn_preprocessed_without_rescale

In [0]:
%%td_hive
insert overwrite table churn_preprocessed_without_rescale
select
  -- add user_id
  row_number() over() as userid

  -- Convert the target variable into binary.
  ,case when is_churn like 'True.' then 1 else 0 end as is_churn
  
  -- Splits into training data and test data
  ,case when rand() < 0.8 then 'train' else 'test' end as class

  -- Categorical features
  ,categorical_features(
    array('state','area_code','intl_plan','vmail_plan'),
    state,area_code,intl_plan,vmail_plan
  ) as q_features

  -- Quantitative features
  ,quantitative_features(
    array('account_length','vmail_massage','day_calls','eve_calls','night_calls','intl_calls','cust_serv_calls','day_mins','eve_mins','night_mins','intl_mins'),
    account_length,vmail_message,day_calls,eve_calls,night_calls,intl_calls,cust_serv_calls,day_mins,eve_mins,night_mins,intl_mins
  ) as c_features
  
  -- Concat categorical features adn quantitative ones. Also, `add_bias` is needed for polynomial models.
  ,add_bias(
    array_concat(

      -- same as 'Categorical features'
      categorical_features(
        array('state','area_code','intl_plan','vmail_plan'),
        state,area_code,intl_plan,vmail_plan
      )
      -- same as 'Qategorical features'
      ,quantitative_features(
        array('account_length','vmail_massage','day_calls','eve_calls','night_calls','intl_calls','cust_serv_calls','day_mins','eve_mins','night_mins','intl_mins'),
        account_length,vmail_message,day_calls,eve_calls,night_calls,intl_calls,cust_serv_calls,day_mins,eve_mins,night_mins,intl_mins
      )
    )
  ) as features
from
  churn_dataset

Check the created table.  
作成したテーブルを確認しましょう。

In [0]:
%%td_presto
select
  *
from
  churn_preprocessed_without_rescale
limit 2

### Build a logistic regression model / ロジスティック回帰モデルの構築

In [0]:
!td table:delete -f $DATABASE churn_logloss_model_without_rescale; td table:create $DATABASE churn_logloss_model_without_rescale

In [0]:
%%td_hive
insert overwrite table churn_logloss_model_without_rescale
select
  train_classifier(
    features,
    is_churn,
    '-loss_function logloss -optimizer SGD -regularization l1'
  ) as (feature, weight)
from
  churn_preprocessed_without_rescale
where
  class like 'train'

Check the positive features.  
プラスに効いている変数を確認しましょう。

In [0]:
%%td_presto
select
  *
from
  churn_logloss_model_without_rescale
order by
  weight desc
limit 5

Check the negative features.  
マイナスに効いている変数を確認しましょう。

In [0]:
%%td_presto
select
  *
from
  churn_logloss_model_without_rescale
order by
  weight asc
limit 5

### Prediction  / 予測

In [0]:
!td table:delete -f $DATABASE churn_prediction_without_rescale; td table:create $DATABASE churn_prediction_without_rescale

In [0]:
%%td_hive
with parsed as (
  select
    userid
    ,is_churn
    ,class
    ,extract_feature(feature) as feature
    ,extract_weight(feature) as value
  from
    churn_preprocessed_without_rescale
  lateral view
    explode(features) t as feature
)

insert overwrite table churn_prediction_without_rescale
select
  userid
  ,is_churn 
  ,class
  ,sigmoid(sum(value * weight)) as prob
from
  parsed
  inner join
  churn_logloss_model_without_rescale model
  on
    parsed.feature = model.feature
group by
  1,2,3

### Evaluation / 評価

Here, we use ROC' auc (= area under curve) as a indication of accuracy.  
[ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) is a chart below.  

The bigger auc is, the better accuracy is.  

<br>

ここでは[ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)のAUCを使って評価します。  
AUCが大きければ大きいほど精度が良いことを表します。

In [0]:
#@title Default title text { display-mode: "code" }
from sklearn import metrics

# Training data
df = td.read_td_query('''
  select
    *
  from
    churn_prediction_without_rescale
  where
    class like 'train'
''', engine=presto)

fig, ax = plt.subplots()
fpr, tpr, thresholds = metrics.roc_curve(df['is_churn'], df['prob'])
train_auc = metrics.auc(fpr, tpr)
ax.plot(fpr, tpr)

# Test data
df = td.read_td_query('''
  select
    *
  from
    churn_prediction_without_rescale
  where
    class like 'test'
''', engine=presto)

fpr, tpr, thresholds = metrics.roc_curve(df['is_churn'], df['prob'])
test_auc = metrics.auc(fpr, tpr)
ax.plot(fpr, tpr)

print("Train auc : {:.4f}\nTest auc : {:.4f}".format(train_auc, test_auc))
plt.show()

### Discussion / 考察

ROC indicates the model do not work well. It may be caused by not having features rescaled.  
Let's re-generate a mode with rescaled feature variables.

<br>

上のROCのAUCを見る限り、うまく学習できていないようです。  
これは説明変数をリスケールしていないことに起因します。リスケールしたモデルを構築してみましょう。

## With rescaling

### Preprocess / 前処理

In [0]:
!td table:delete -f $DATABASE churn_preprocessed_with_rescale; td table:create $DATABASE churn_preprocessed_with_rescale

#### Calculate the minimum and maximum of each columns / 各カラムの最小値・最大値を計算する

In [0]:
%%td_presto
select 'account_length' as column, min(account_length) as min, max(account_length) as max from churn_dataset
union all
select 'vmail_message' as column, min(vmail_message) as min, max(vmail_message) as max from churn_dataset
union all
select 'day_calls' as column, min(day_calls) as min, max(day_calls) as max from churn_dataset
union all
select 'eve_calls' as column, min(eve_calls) as min, max(eve_calls) as max from churn_dataset
union all
select 'night_calls' as column, min(night_calls) as min, max(night_calls) as max from churn_dataset
union all
select 'intl_calls' as column, min(intl_calls) as min, max(intl_calls) as max from churn_dataset
union all
select 'cust_serv_calls' as column, min(cust_serv_calls) as min, max(cust_serv_calls) as max from churn_dataset
union all
select 'day_mins' as column, min(day_mins) as min, max(day_mins) as max from churn_dataset
union all
select 'eve_mins' as column, min(eve_mins) as min, max(eve_mins) as max from churn_dataset
union all
select 'night_mins' as column, min(night_mins) as min, max(night_mins) as max from churn_dataset
union all
select 'intl_mins' as column, min(intl_mins) as min, max(intl_mins) as max from churn_dataset

#### Rescale with results above / リスケールする

In [0]:
%%td_hive
with rescaled as (
  select
    is_churn,
    state,area_code,intl_plan,vmail_plan,    

    rescale(vmail_message, 0, 51) as vmail_message,
    rescale(intl_calls, 0, 20) as intl_calls,
    rescale(cust_serv_calls, 0, 9) as cust_serv_calls,
    rescale(eve_calls, 0, 170) as eve_calls,
    rescale(account_length, 1, 243) as account_length,
    rescale(night_calls, 33, 175) as night_calls,
    rescale(day_mins, 0, 350.8) as day_mins,
    rescale(day_calls, 0, 165) as day_calls,
    rescale(eve_mins, 0, 363.7) as eve_mins,
    rescale(intl_mins, 0, 20) as intl_mins,
    rescale(night_mins, 23.2, 395) as night_mins

  from
    churn_dataset
)

insert overwrite table churn_preprocessed_with_rescale
select
  -- add user_id
  row_number() over() as userid

  -- Convert the target variable into binary.
  ,case when is_churn like 'True.' then 1 else 0 end as is_churn
  
  -- Splits into training data and test data
  ,case when rand() < 0.8 then 'train' else 'test' end as class
  
  -- Concat categorical features adn quantitative ones. Also, `add_bias` is needed for polynomial models.
  ,add_bias(
    array_concat(

      -- same as 'Categorical features'
      categorical_features(
        array('state','area_code','intl_plan','vmail_plan'),
        state,area_code,intl_plan,vmail_plan
      )
      -- same as 'Qategorical features'
      ,quantitative_features(
        array('account_length','vmail_massage','day_calls','eve_calls','night_calls','intl_calls','cust_serv_calls','day_mins','eve_mins','night_mins','intl_mins'),
        account_length,vmail_message,day_calls,eve_calls,night_calls,intl_calls,cust_serv_calls,day_mins,eve_mins,night_mins,intl_mins
      )
    )
  ) as features
from
  rescaled

### Build a model, again / モデル再構築

In [0]:
!td table:delete -f $DATABASE churn_logloss_model_rescale; td table:create $DATABASE churn_logloss_model_rescale

In [0]:
%%td_hive
insert overwrite table churn_logloss_model_rescale
select
  train_classifier(
    features,
    is_churn,
    '-loss_function logloss -optimizer SGD -regularization l1'
  ) as (feature, weight)
from
  churn_preprocessed_with_rescale
where
  class like 'train'

### Prediction / 予測

In [0]:
!td table:delete -f $DATABASE churn_prediction_with_rescale; td table:create $DATABASE churn_prediction_with_rescale

In [0]:
%%td_hive
with parsed as (
  select
    userid
    ,is_churn
    ,class
    ,extract_feature(feature) as feature
    ,extract_weight(feature) as value
  from
    churn_preprocessed_with_rescale
  lateral view
    explode(features) t as feature
)

insert overwrite table churn_prediction_with_rescale 
select
  userid
  ,is_churn 
  ,class
  ,sigmoid(sum(value * weight)) as prob
from
  parsed
  inner join
  churn_logloss_model_rescale model
  on
    parsed.feature = model.feature
group by
  1,2,3

### Evaluation / 評価

In [0]:
from sklearn import metrics

# Training data
df = td.read_td_query('''
  select
    *
  from
    churn_prediction_with_rescale
  where
    class like 'train'
''', engine=presto)

fig, ax = plt.subplots()
fpr, tpr, thresholds = metrics.roc_curve(df['is_churn'], df['prob'])
train_auc = metrics.auc(fpr, tpr)
ax.plot(fpr, tpr)

# Test data
df = td.read_td_query('''
  select
    *
  from
    churn_prediction_with_rescale
  where
    class like 'test'
''', engine=presto)

fpr, tpr, thresholds = metrics.roc_curve(df['is_churn'], df['prob'])
test_auc = metrics.auc(fpr, tpr)
ax.plot(fpr, tpr)

print("Train auc : {:.4f}\nTest auc : {:.4f}".format(train_auc, test_auc))
plt.show()

### Discussion / 考察

Having features rescaled, the accuracy got better.

In order to improve the accuracy, 
- add/remove features.
- aggregate existing features.
- have the number of target classes balanced.
- try other models rather than logistic regression.
- and so on.

<br>

リスケールすることで精度を向上できた。  

さらに精度を上げるためには以下が考えられる。
- 説明変数の追加/削除
- 変数の集約
- 正例/負例の数を等しくする
- ロジスティック回帰以外のモデルを試す

# Appendix

## AUC of ROC

AUC of ROC is one of accuracy indications of binary classification tasks.

When probabilities are randomized, the auc of ROC is around 0.50   
and the ROC is nealy  `y = x`.
The better machine learning model works, the bigger AUC of ROC gets.  
The maximum value is 1.0.

<br>

ROCのAUCは二値分類の精度指標のひとつです。  
推定した値が乱数であれば、AUCの値は約0.5になり、ROCのグラフは `y = x` に近くなります。  
(AUC = Area Under Curve はグラフより下の部分の面積を指します。)  
精度の良いモデルであればあるほど、AUCは大きくなります。
最大値は1.0です。

In [0]:
df = td.read_td_query('''
  select
    case when is_churn like 'True.' then 1 else 0 end as is_churn
    ,rand() as prob
  from
    churn_dataset
''', engine=presto)

# Plot ROC
fpr, tpr, thresholds = metrics.roc_curve(df['is_churn'] == 1, df['prob'])
print('ROC of AUC : {:.4f}'.format(metrics.auc(fpr, tpr)))
plt.plot(fpr, tpr)
plt.show()